In [1]:
# Imports

# Libraries
import uproot
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import lz4
import xxhash    
import math
import seaborn as sns

# Local stuff
from branches import *
from panda_helpers import *
from helpers import *

In [2]:
# Define the path to the file you want to open

# Make sure it is a __flat__ CAF file!!!
filename = "/Users/sxy1439/Documents/sbnd/CAFana/test_caf.root"
filename

'/Users/sxy1439/Documents/sbnd/CAFana/test_caf.root'

In [3]:
tname = "recTree"
events = uproot.open(filename + ":" + tname)

In [4]:
#events

In [5]:
nudf = events.arrays(mcbranches, library="pd")
#nudf

In [6]:
# Make a plot!
#_ = plt.hist(nudf["rec.mc.nu.E"])
#plt.xlabel("Neutrino Energy [GeV]")
#plt.ylabel("Entries")

In [7]:
# In order to handle nested data, I have a helper function that parses the FLAT CAF structure

# delete the old nudf
del nudf
nudf = loadbranches(events, mcbranches)
slcdf = loadbranches(events, slcbranches)
trkdf = loadbranches(events, trkbranches)

In [8]:
nudf

rec.mc.nu.E  rec.mc.nu.position.x  \
entry rec.mc.nu..index                                      
0     0                    0.940681           -584.593567   
      1                    0.896295              8.005111   
1     0                    3.956313            -68.982361   
      1                    1.277367           -208.679321   
      2                    0.395579            474.114868   
...                             ...                   ...   
2767  1                    0.691947           -156.695847   
2768  0                    1.208528           -199.727173   
2769  0                    0.722863            123.196190   
2770  0                    0.716736            156.997635   
2771  0                    1.269023            125.427917   

                        rec.mc.nu.position.y  rec.mc.nu.position.z  \
entry rec.mc.nu..index                                               
0     0                           245.041122            229.196503   
      1                            -7.561608            359.866394   
1     0                          -140.322510            512.446350   
      1                            35.178295           -473.911743   
      2                           313.314972             11.619415   
...                                      ...                   ...   
2767  1                           -82.170418             37.647198   
2768  0                             5.899607            479.121429   
2769  0                           140.274200            169.813690   
2770  0                           107.968300            360.384064   
2771  0                           -58.416622            -25.419466   

                        rec.mc.nu.pdg  rec.mc.nu.iscc  rec.mc.nu.momentum.x  
entry rec.mc.nu..index                                                       
0     0                            14               1             -0.046415  
      1                            14               0              0.004911  
1     0                            14               1             -0.013122  
      1                            14               1             -0.024382  
      2                            14               1              0.019968  
...                               ...             ...                   ...  
2767  1                            14               1             -0.006495  
2768  0                            14               1             -0.015370  
2769  0                            14               1              0.013606  
2770  0                            14               0              0.015345  
2771  0                            14               1              0.027089  

[4974 rows x 7 columns]

In [9]:
slcdf

rec.slc.charge  rec.slc.vertex.x  rec.slc.vertex.y  \
entry rec.slc..index                                                       
0     0                 3.402823e+38         79.662437        -16.618870   
      1                 3.402823e+38         16.960104        -16.154673   
      2                 3.402823e+38        -23.143448        203.333801   
      3                 3.402823e+38        -12.372738        178.746170   
      4                 3.402823e+38        -51.440487        204.688461   
...                              ...               ...               ...   
2770  4                 3.402823e+38         56.233780       -203.076675   
2771  0                 3.402823e+38        140.761017        -85.549110   
      1                 3.402823e+38        118.967911        182.477005   
      2                 3.402823e+38        -29.728001        205.006088   
      3                 3.402823e+38       -232.862213         -3.761062   

                      rec.slc.vertex.z  rec.slc.self  rec.slc.tmatch.eff  \
entry rec.slc..index                                                       
0     0                       3.645125            41                 NaN   
      1                     373.886505            40            0.559702   
      2                     295.167572             0                 NaN   
      3                     510.530029             1                 NaN   
      4                     464.692261             3                 NaN   
...                                ...           ...                 ...   
2770  4                     331.231476             4                 NaN   
2771  0                       4.780899            15            0.958125   
      1                     343.298370            14                 NaN   
      2                     143.695679             0                 NaN   
      3                     171.446243             1                 NaN   

                      rec.slc.tmatch.pur  rec.slc.tmatch.index  \
entry rec.slc..index                                             
0     0                              NaN                  -999   
      1                         0.916363                     1   
      2                              NaN                  -999   
      3                              NaN                  -999   
      4                              NaN                  -999   
...                                  ...                   ...   
2770  4                              NaN                  -999   
2771  0                         0.975834                     0   
      1                              NaN                  -999   
      2                              NaN                  -999   
      3                              NaN                  -999   

                      rec.slc.producer  rec.slc.nu_score  \
entry rec.slc..index                                       
0     0                              0          0.471248   
      1                              0          0.628609   
      2                              0         -1.000000   
      3                              0         -1.000000   
      4                              0         -1.000000   
...                                ...               ...   
2770  4                              0         -1.000000   
2771  0                              0          0.432571   
      1                              0          0.484329   
      2                              0         -1.000000   
      3                              0         -1.000000   

                      rec.slc.truth.momentum.x  rec.slc.truth.momentum.y  \
entry rec.slc..index                                                       
0     0                                    NaN                       NaN   
      1                               0.004911                 -0.001253   
      2                                    NaN                       NaN   
      3                 

In [31]:
# Now, we are going to a quick bit of the numu Event selection.

# First, we can look at some reconstructed variables and compare them for neutrino-slices and cosmic-slices.
# To do this, we need to merge the neutrino data frame into the slice data frame using truth matching.

# We can do this with a "merge"

# Cut on the truth matching -- require the slice contains more than half of the deposited neutrino energy.
# This ensures that each neutrino can only have one reconstructed slice match
#slc_has_nu_match = slcdf["rec.slc.tmatch.eff"] > 0.5
#print(slc_has_nu_match)
#print(np.invert(slc_has_nu_match))

# Ignore index matches where the efficiency cut fails
#slc_has_nu_match = slcdf.loc[(slcdf["rec.slc.tmatch.eff"] > 0.5)]
slc_has_nu_match = (slcdf["rec.slc.tmatch.eff"] > 0.5)                            
slcdf.loc[np.invert(slc_has_nu_match) & (slcdf["rec.slc.tmatch.index"] >= 0), "rec.slc.tmatch.index"] = np.nan

matchdf = pd.merge(slcdf.reset_index(), # Merging can mess with the multi-index -- we'll fix this later
                 nudf.reset_index(),
                 left_on=["entry", "rec.slc.tmatch.index"], # Match on the entry number than the neutrino index
                 right_on=["entry", "rec.mc.nu..index"], 
                 how="left", # Keep every slice
                 )

matchdf = matchdf.set_index(["entry", "rec.slc..index"], verify_integrity=True)

# check that each neutrino matches to only one slice
assert(matchdf.groupby(["entry", "rec.mc.nu..index"])["rec.slc.charge"].count().max() == 1)

match_has_nu_match = matchdf.loc[(matchdf["rec.slc.tmatch.eff"] > 0.5) & (matchdf["rec.slc.tmatch.index"] >= 0)]
#selSlc_max_eff = matchdf.groupby(["entry"]).loc[matchdf["rec.slc.tmatch.eff"].idxmax()]
selSlc_grp = matchdf.groupby(["entry", "rec.slc..index"])
#selSlc_max_eff = selSlc_grp.loc["rec.slc.tmatch.eff"].idxmax()                         
print(selSlc_grp)
#print(matchdf.head(10))

In [ ]:
#trkdf

In [ ]:
#matchdf1 = pd.merge(slcdf.reset_index(), # Merging can mess with the multi-index -- we'll fix this later
#                 nudf.reset_index(),
#                 left_on=["entry", "rec.slc.fmatch."], # Match on the entry number than the neutrino index
#                 right_on=["entry", "rec.mc.nu..index"], 
#                 how="left", # Keep every slice
#                 )
#
#matchdf = matchdf.set_index(["entry", "rec.slc..index"], verify_integrity=True)
#
# check that each neutrino matches to only one slice
#assert(matchdf.groupby(["entry", "rec.mc.nu..index"])["rec.slc.charge"].count().max() == 1)

In [ ]:
trkdf["rec.slc.reco.trk.truth.p.pdg"]

In [ ]:
#matchdf

In [ ]:
# Energy of neutrinos with a matched slice!!

# NOTE: one thing to keep in mind -- merge's make a cut on the physics

_ = plt.hist(matchdf["rec.mc.nu.E"])

In [ ]:
# Now we can look at an example slice variable -- the Pandora "nu" score

var = matchdf["rec.slc.nu_score"]

is_numu_cc = (np.abs(matchdf["rec.mc.nu.pdg"]) == 14) & (matchdf["rec.mc.nu.iscc"])
is_cosmic = (matchdf["rec.slc.tmatch.index"] < 0)
is_nuel_cc = (np.abs(matchdf["rec.mc.nu.pdg"]) == 12) & (matchdf["rec.mc.nu.iscc"])
is_nc =  (matchdf["rec.mc.nu.iscc"]==0)
bins=np.linspace(-1.1, 1.1, 44)
_ = plt.hist(var[is_numu_cc], bins=bins, histtype="step", label=r"$\nu_\mu$ CC")
#_ = plt.hist(var[is_cosmic], bins=bins, histtype="step", label="Cosmic")
_ = plt.hist(var[is_nuel_cc], bins=bins, histtype="step", label=r"$\nu_e$ CC")
_ = plt.hist(var[is_nc], bins=bins, histtype="step", label=r"$\nu$ NC")
_ = plt.legend()
_ = plt.xlabel(r"Pandora $\nu$ Score")

In [ ]:
is_numu_cc = (np.abs(matchdf["rec.mc.nu.pdg"]) == 14) & (matchdf["rec.mc.nu.iscc"])

In [ ]:
####################### df 2 (dataframe without any cut on L) ###############################################

In [ ]:
# Event selection (with no cuts on L)

# Using the algorithm from:
# https://sbn-docdb.fnal.gov/cgi-bin/private/RetrieveFile?docid=20139&filename=event_selection.pdf&version=2

# Now, using the trkdf
trkdf["trk_contained"] =\
    InFV(trkdf["rec.slc.reco.trk.start.x"], trkdf["rec.slc.reco.trk.start.y"], trkdf["rec.slc.reco.trk.start.z"]) &\
    InFV(trkdf["rec.slc.reco.trk.end.x"], trkdf["rec.slc.reco.trk.end.y"], trkdf["rec.slc.reco.trk.end.z"])

primary_track_2 = trkdf\
    .sort_values(["entry", "rec.slc..index", 'rec.slc.reco.trk.len'], ascending=[True, True, False])\
    .groupby(["entry", "rec.slc..index"]).head(5)

print(trkdf.head(50))
#primary_track_2["isCount"] = primary_track_2["trk_contained"].count()
#display(primary_track_2)
#print(primary_track_2.head(20))

#trkdf = trkdf.apply(trkdf["trk_contained"] == False)
  
# Count number of True in the series
#num_rows = len(trkdf[trkdf == True].index)

#if (trkdf["trk_contained"]==False).all():
#    dataframevalue.drop(trkdf["rec.slc..index"])

#primary_track_2.filter(lambda g: (g["trk_contained"]).all())

#num_rows = len(primary_track_2.index)
#print(num_rows)

# check valid chi2 -- just look at collection plane for now
muon_chi2 = (trkdf["rec.slc.reco.trk.chi2pid.2.chi2_muon"] < 30.) &\
    (trkdf["rec.slc.reco.trk.chi2pid.2.chi2_proton"] > 60.)

# Valid primary track candidates
#primary_track_candidate = (trkdf["trk_contained"] & muon_chi2 & (trkdf["rec.slc.reco.trk.len"] > 50.)) |\
#        (trkdf["rec.slc.reco.trk.len"] > 100.)

primary_track_candidate_2 = (trkdf["trk_contained"] & muon_chi2 ) |\
        (trkdf["rec.slc.reco.trk.len"] > 100.)
#primary_track_candidate_2 = (trkdf["trk_contained"]) |\
#        (trkdf["rec.slc.reco.trk.len"] > 100.)
    
### Binary OR(|) Operator

#primary_track_2 = trkdf[primary_track_candidate_2]\
#    .sort_values(["entry", "rec.slc..index", 'rec.slc.reco.trk.len'], ascending=[True, True, False])\
#    .groupby(["entry", "rec.slc..index"]).head(5)
#print(primary_track_2.head(20))
#print(primary_track_candidate_2.head(15))
#print(trkdf["trk_contained"])
#print(primary_track_2["rec.slc.reco.trk..index"])

In [ ]:
# Now, merge the primary track into the slice df : with no cuts on L
df_2 = pd.merge(matchdf.reset_index(), primary_track_2,
              left_on=["entry", "rec.slc..index"], # match on spill then slice number
              right_on=["entry", "rec.slc..index"],
              how="inner", # only keep slices with a primary track
              validate="one_to_one", # Always validate when you can! Don't put two primary tracks in a slice -- this would double-count a slice
             )
#print(df)

In [ ]:
####### Plotting #events with momentum (Contained) : with no cuts on L ########



# Now we can compute more stuff! Like the primary track momentum
###### calculating muon momentum (contained) :: Reco ############
is_numu_cc = (np.abs(df_2["rec.mc.nu.pdg"]) == 14) & (df_2["rec.mc.nu.iscc"])
is_proton = df_2["rec.slc.reco.trk.truth.p.pdg"] == 2212
is_neutron = df_2["rec.slc.reco.trk.truth.p.pdg"] == 2112
is_pi_pm = np.abs(df_2["rec.slc.reco.trk.truth.p.pdg"]) == 211
is_pi0 = df_2["rec.slc.reco.trk.truth.p.pdg"] == 111
is_elec_prot = df_2["rec.slc.reco.trk.truth.p.pdg"] == 11      #### electron positron
is_muon_antim = df_2["rec.slc.reco.trk.truth.p.pdg"] == 13
is_gamma = df_2["rec.slc.reco.trk.truth.p.pdg"] == 22
#is_cosmic = (df_2["rec.slc.tmatch.index"] < 0)

#px = df["rec.slc.truth.momentum.x"]
#py = df["rec.slc.truth.momentum.y"]
#pz = df["rec.slc.truth.momentum.z"]

#momentum_square= px**2 + py**2 + pz**2
#total_momentum = np.sqrt(momentum_square)


var = df_2["rec.slc.reco.trk.rangeP.p_muon"]
#var = df["rec.slc.truth.momentum.x"] + df["rec.slc.truth.momentum.y"] + df["rec.slc.truth.momentum.z"]

plt.figure()
bins = np.linspace(0, 2, 25)
#(vt1, bt1, pt1) = plt.hist(var[is_numu_cc & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$nu_mu$ CC")
#(v11, b11, p11) = plt.hist(var[is_proton & is_numu_cc & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"proton", stacked=True)
#(v21, b21, p21) = plt.hist(var[is_neutron & is_numu_cc & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"neutron", stacked=True)
#(v31, b31, p31) = plt.hist(var[is_pi_pm & is_numu_cc & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\pi+-$", stacked=True)
#(v41, b41, p41) = plt.hist(var[is_pi0 & is_numu_cc & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\pi^{0}$", stacked=True)
#(v51, b51, p51) = plt.hist(var[is_elec_prot & is_numu_cc & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"e+-", stacked=True)
#(v61, b61, p61)= plt.hist(var[is_muon_antim & is_numu_cc & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\mu+-$", stacked=True)
#(v71, b71, p71) = plt.hist(var[is_gamma & is_numu_cc & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\gamma$", stacked=True)

#_ = plt.hist(var[is_cosmic & df["trk_contained"]], bins=bins, histtype="step", label="Cosmic")

aa = var[is_proton & is_numu_cc & df_2["trk_contained"] ]
bb = var[is_neutron & is_numu_cc & df_2["trk_contained"] ]
cc = var[is_pi_pm & is_numu_cc & df_2["trk_contained"] ]
dd = var[is_pi0 & is_numu_cc & df_2["trk_contained"] ]
ee = var[is_elec_prot & is_numu_cc & df_2["trk_contained"] ]
ff = var[is_muon_antim & is_numu_cc & df_2["trk_contained"] ]
gg = var[is_gamma & is_numu_cc & df_2["trk_contained"] ]

colors = ['olivedrab', 'cyan', 'steelblue', 'magenta', 'yellow', 'tomato' ,'green']
names = ['proton', 'neutron', '$\pi+-$', '$\pi^{0}$', 'e+-', '$\mu+-$', '$\gamma$']
plt.hist([aa, bb, cc, dd, ee, ff, gg], color=colors, bins = bins, label=names, density=False, stacked = True)

plt.legend()
plt.xlabel("Reco Muon Momentum (Contained) with no cut on L - per particle type [GeV/c]")
plt.ylabel("# events")
plt.title("Plot wrt reco muon momentum with no cut on L")
plt.show()



In [ ]:
####### Plotting #events with momentum (Contained) : with no cuts on L ########



# Now we can compute more stuff! Like the primary track momentum
###### calculating muon momentum (contained) :: Reco ############
is_numu_cc = (np.abs(df_2["rec.mc.nu.pdg"]) == 14) & (df_2["rec.mc.nu.iscc"])
is_proton = df_2["rec.slc.reco.trk.truth.p.pdg"] == 2212
is_neutron = df_2["rec.slc.reco.trk.truth.p.pdg"] == 2112
is_pi_pm = np.abs(df_2["rec.slc.reco.trk.truth.p.pdg"]) == 211
is_pi0 = df_2["rec.slc.reco.trk.truth.p.pdg"] == 111
is_elec_prot = df_2["rec.slc.reco.trk.truth.p.pdg"] == 11      #### electron positron
is_muon_antim = df_2["rec.slc.reco.trk.truth.p.pdg"] == 13
is_gamma = df_2["rec.slc.reco.trk.truth.p.pdg"] == 22
#is_cosmic = (df_2["rec.slc.tmatch.index"] < 0)

#px = df["rec.slc.truth.momentum.x"]
#py = df["rec.slc.truth.momentum.y"]
#pz = df["rec.slc.truth.momentum.z"]

#momentum_square= px**2 + py**2 + pz**2
#total_momentum = np.sqrt(momentum_square)


var = df_2["rec.slc.reco.trk.rangeP.p_muon"]
#var = df["rec.slc.truth.momentum.x"] + df["rec.slc.truth.momentum.y"] + df["rec.slc.truth.momentum.z"]

plt.figure()
bins = np.linspace(0, 2, 25)
(vt1, bt1, pt1) = plt.hist(var[is_numu_cc & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\nu_\mu$ CC")
(v11, b11, p11) = plt.hist(var[is_proton & is_numu_cc & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"proton", stacked=True)
(v21, b21, p21) = plt.hist(var[is_neutron & is_numu_cc & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"neutron", stacked=True)
(v31, b31, p31) = plt.hist(var[is_pi_pm & is_numu_cc & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\pi+-$", stacked=True)
(v41, b41, p41) = plt.hist(var[is_pi0 & is_numu_cc & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\pi^{0}$", stacked=True)
(v51, b51, p51) = plt.hist(var[is_elec_prot & is_numu_cc & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"e+-", stacked=True)
(v61, b61, p61)= plt.hist(var[is_muon_antim & is_numu_cc & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\mu+-$", stacked=True)
(v71, b71, p71) = plt.hist(var[is_gamma & is_numu_cc & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\gamma$", stacked=True)

#_ = plt.hist(var[is_cosmic & df["trk_contained"]], bins=bins, histtype="step", label="Cosmic")
plt.legend()
plt.xlabel("Reco Muon Momentum (Contained) with no cut on L - per particle type [GeV/c]")
plt.ylabel("# events")
plt.show()


In [ ]:
#### plotting purity , purity = np.divide(selected event that are true (S & T), total selected events) #####
######### with no cut on L ######
np.seterr(invalid='ignore')
rat11 = np.divide(v11,vt1)
rat21 = np.divide(v21,vt1)
rat31 = np.divide(v31,vt1)
rat41 = np.divide(v41,vt1)
rat51 = np.divide(v51,vt1)
rat61 = np.divide(v61,vt1)
rat71 = np.divide(v71,vt1)
cbins1   = 0.5*(b11[:-1] + b11[1:])
plt.plot(cbins1,rat11,label='proton')
plt.plot(cbins1,rat21,label='neutron')
plt.plot(cbins1,rat31,label='charged pion')
plt.plot(cbins1,rat41,label='pion 0')
plt.plot(cbins1,rat51,label='electron/po')
plt.plot(cbins1,rat61,label='mu_antimu')
plt.plot(cbins1,rat71,label='gamma')
plt.legend()
plt.xlabel("Reco muon momentum (centre of the bin)")
plt.ylabel("Purity (ratio of selected true and total selected)")
plt.title("Purity plot for no L cut")

In [ ]:
####### Plotting #events with Length (Contained) with no cut on L ########



# Now we can compute more stuff! Like the primary track momentum
###### calculating muon momentum (contained) :: Reco ############
is_numu_cc = (np.abs(df_2["rec.mc.nu.pdg"]) == 14) & (df_2["rec.mc.nu.iscc"])
is_proton = df_2["rec.slc.reco.trk.truth.p.pdg"] == 2212
is_neutron = df_2["rec.slc.reco.trk.truth.p.pdg"] == 2112
is_pi_pm = np.abs(df_2["rec.slc.reco.trk.truth.p.pdg"]) == 211
is_pi0 = df_2["rec.slc.reco.trk.truth.p.pdg"] == 111
is_elec_prot = df_2["rec.slc.reco.trk.truth.p.pdg"] == 11      #### electron positron
is_muon_antim = df_2["rec.slc.reco.trk.truth.p.pdg"] == 13
is_gamma = df_2["rec.slc.reco.trk.truth.p.pdg"] == 22
#is_cosmic = (df_2["rec.slc.tmatch.index"] < 0)

#delx = df_2["rec.slc.reco.trk.end.x"] - df_2["rec.slc.reco.trk.start.x"]
#dely = df_2["rec.slc.reco.trk.end.y"] - df_2["rec.slc.reco.trk.start.y"]
#delz = df_2["rec.slc.reco.trk.end.z"] - df_2["rec.slc.reco.trk.start.z"]

#length = (((delx**2)+(dely**2)+(delz**2))**(0.5))

var = df_2["rec.slc.reco.trk.len"]
#var = df["rec.slc.truth.momentum.x"] + df["rec.slc.truth.momentum.y"] + df["rec.slc.truth.momentum.z"]


#bins = np.linspace(0, 2, 25)
#_ = plt.hist(var[is_numu_cc & df["trk_contained"] ], bins=bins, histtype="step", label=r"$\nu_\mu$ CC")
_ = plt.hist(var[is_proton & is_numu_cc & df_2["trk_contained"] ], histtype="step", label=r"proton", stacked=True)
_ = plt.hist(var[is_neutron & is_numu_cc & df_2["trk_contained"] ], histtype="step", label=r"neutron", stacked=True)
_ = plt.hist(var[is_pi_pm & is_numu_cc & df_2["trk_contained"] ], histtype="step", label=r"$\pi+-$", stacked=True)
_ = plt.hist(var[is_pi0 & is_numu_cc & df_2["trk_contained"] ], histtype="step", label=r"$\pi^{0}$", stacked=True)
_ = plt.hist(var[is_elec_prot & is_numu_cc & df_2["trk_contained"] ], histtype="step", label=r"e+-", stacked=True)
_ = plt.hist(var[is_muon_antim & is_numu_cc & df_2["trk_contained"] ], histtype="step", label=r"$\mu+-$", stacked=True)
_ = plt.hist(var[is_gamma & is_numu_cc & df_2["trk_contained"] ], histtype="step", label=r"$\gamma$", stacked=True)

#_ = plt.hist(var[is_cosmic & df["trk_contained"]], bins=bins, histtype="step", label="Cosmic")
plt.legend()
plt.xlabel("Reco track length (Contained) with no cut on L - per particle type [cm]")
plt.ylabel("# events")
plt.show()

In [ ]:
####### Plotting #events with Length (Contained) with no cut on L ########



# Now we can compute more stuff! Like the primary track momentum
###### calculating muon momentum (contained) :: Reco ############
is_numu_cc = (np.abs(df_2["rec.mc.nu.pdg"]) == 14) & (df_2["rec.mc.nu.iscc"])
is_proton = df_2["rec.slc.reco.trk.truth.p.pdg"] == 2212
is_neutron = df_2["rec.slc.reco.trk.truth.p.pdg"] == 2112
is_pi_pm = np.abs(df_2["rec.slc.reco.trk.truth.p.pdg"]) == 211
is_pi0 = df_2["rec.slc.reco.trk.truth.p.pdg"] == 111
is_elec_prot = df_2["rec.slc.reco.trk.truth.p.pdg"] == 11      #### electron positron
is_muon_antim = df_2["rec.slc.reco.trk.truth.p.pdg"] == 13
is_gamma = df_2["rec.slc.reco.trk.truth.p.pdg"] == 22
#is_cosmic = (df_2["rec.slc.tmatch.index"] < 0)

delx = df_2["rec.slc.reco.trk.end.x"] - df_2["rec.slc.reco.trk.start.x"]
dely = df_2["rec.slc.reco.trk.end.y"] - df_2["rec.slc.reco.trk.start.y"]
delz = df_2["rec.slc.reco.trk.end.z"] - df_2["rec.slc.reco.trk.start.z"]

length = (((delx**2)+(dely**2)+(delz**2))**(0.5))

var = df_2["rec.slc.reco.trk.len"]
#var = df["rec.slc.truth.momentum.x"] + df["rec.slc.truth.momentum.y"] + df["rec.slc.truth.momentum.z"]


#bins = np.linspace(0, 2, 25)
#_ = plt.hist(var[is_numu_cc & df["trk_contained"] ], bins=bins, histtype="step", label=r"$\nu_\mu$ CC")
#_ = plt.hist(var[is_proton & is_numu_cc & df_2["trk_contained"] ], histtype="step", label=r"proton", stacked=True)
#_ = plt.hist(var[is_neutron & is_numu_cc & df_2["trk_contained"] ], histtype="step", label=r"neutron", stacked=True)
#_ = plt.hist(var[is_pi_pm & is_numu_cc & df_2["trk_contained"] ], histtype="step", label=r"$\pi+-$", stacked=True)
#_ = plt.hist(var[is_pi0 & is_numu_cc & df_2["trk_contained"] ], histtype="step", label=r"$\pi^{0}$", stacked=True)
#_ = plt.hist(var[is_elec_prot & is_numu_cc & df_2["trk_contained"] ], histtype="step", label=r"e+-", stacked=True)
#_ = plt.hist(var[is_muon_antim & is_numu_cc & df_2["trk_contained"] ], histtype="step", label=r"$\mu+-$", stacked=True)
#_ = plt.hist(var[is_gamma & is_numu_cc & df_2["trk_contained"] ], histtype="step", label=r"$\gamma$", stacked=True)

aa = var[is_proton & is_numu_cc & df_2["trk_contained"] ]
bb = var[is_neutron & is_numu_cc & df_2["trk_contained"] ]
cc = var[is_pi_pm & is_numu_cc & df_2["trk_contained"] ]
dd = var[is_pi0 & is_numu_cc & df_2["trk_contained"] ]
ee = var[is_elec_prot & is_numu_cc & df_2["trk_contained"] ]
ff = var[is_muon_antim & is_numu_cc & df_2["trk_contained"] ]
gg = var[is_gamma & is_numu_cc & df_2["trk_contained"] ]

colors = ['olivedrab', 'cyan', 'steelblue', 'magenta', 'yellow', 'tomato' ,'green']
names = ['proton', 'neutron', '$\pi+-$', '$\pi^{0}$', 'e+-', '$\mu+-$', '$\gamma$']
plt.hist([aa, bb, cc, dd, ee, ff, gg], color=colors, label=names, density=False, stacked = True)


#_ = plt.hist(var[is_cosmic & df["trk_contained"]], bins=bins, histtype="step", label="Cosmic")
plt.legend()
plt.xlabel("Reco track length (Contained) with no cut on L - per particle type [cm]")
plt.ylabel("# events")
plt.title("Plot wrt reco trk length with no cut on L")
plt.show()

In [ ]:
########### 2D plot with no cut on L , x_axis = muon momentum, y_axis = track length ###################

plt.clf()
VtxXY=sns.jointplot(data=df_2, x="rec.slc.reco.trk.rangeP.p_muon",y=length, kind = 'hist', bins=100)
VtxXY.set_axis_labels('Reco muon momentum (GeV/c)', 'Reco track length (cm)')
#plt.savefig('plots/GiBUU_VXY-NuMu.pdf')

In [ ]:
######### looking for difference between Momentum by range and multiple coulomb scattering (MCS) #############

In [ ]:
####### Plotting #events with momentum (Contained) : with no cuts on L ########



# Now we can compute more stuff! Like the primary track momentum
###### calculating muon momentum (contained) :: Reco ############
is_numu_cc = (np.abs(df_2["rec.mc.nu.pdg"]) == 14) & (df_2["rec.mc.nu.iscc"])
is_proton = df_2["rec.slc.reco.trk.truth.p.pdg"] == 2212
is_neutron = df_2["rec.slc.reco.trk.truth.p.pdg"] == 2112
is_pi_pm = np.abs(df_2["rec.slc.reco.trk.truth.p.pdg"]) == 211
is_pi0 = df_2["rec.slc.reco.trk.truth.p.pdg"] == 111
is_elec_prot = df_2["rec.slc.reco.trk.truth.p.pdg"] == 11      #### electron positron
is_muon_antim = df_2["rec.slc.reco.trk.truth.p.pdg"] == 13
is_gamma = df_2["rec.slc.reco.trk.truth.p.pdg"] == 22
#is_cosmic = (df_2["rec.slc.tmatch.index"] < 0)


var_Prange = df_2["rec.slc.reco.trk.rangeP.p_muon"]
var_mcs = df_2["rec.slc.reco.trk.mcsP.fwdP_muon"]
#var = df["rec.slc.truth.momentum.x"] + df["rec.slc.truth.momentum.y"] + df["rec.slc.truth.momentum.z"]

plt.figure()
bins = np.linspace(0, 2, 25)
#_ = plt.hist(var[is_numu_cc & df["trk_contained"] ], bins=bins, histtype="step", label=r"$\nu_\mu$ CC")
#_ = plt.hist(var_Prange[is_proton & is_numu_cc & df_2["trk_contained"] ] - var_mcs[is_proton & is_numu_cc & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"proton", stacked=True)
#_ = plt.hist(var_Prange[is_neutron & is_numu_cc & df_2["trk_contained"] ] - var_mcs[is_neutron & is_numu_cc & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"neutron", stacked=True)
#_ = plt.hist(var_Prange[is_pi_pm & is_numu_cc & df_2["trk_contained"] ] - var_mcs[is_pi_pm & is_numu_cc & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\pi+-$", stacked=True)
#_ = plt.hist(var_Prange[is_pi0 & is_numu_cc & df_2["trk_contained"] ] - var_mcs[is_pi0 & is_numu_cc & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\pi^{0}$", stacked=True)
#_ = plt.hist(var_Prange[is_elec_prot & is_numu_cc & df_2["trk_contained"] ] - var_mcs[is_elec_prot & is_numu_cc & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"e+-", stacked=True)
#_ = plt.hist(var_Prange[is_muon_antim & is_numu_cc & df_2["trk_contained"] ] - var_mcs[is_muon_antim & is_numu_cc & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\mu+-$", stacked=True)
#_ = plt.hist(var_Prange[is_gamma & is_numu_cc & df_2["trk_contained"] ] - var_mcs[is_gamma & is_numu_cc & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\gamma$", stacked=True)


aa = var_Prange[is_proton & is_numu_cc & df_2["trk_contained"] ] - var_mcs[is_proton & is_numu_cc & df_2["trk_contained"] ]
bb = var_Prange[is_neutron & is_numu_cc & df_2["trk_contained"] ] - var_mcs[is_neutron & is_numu_cc & df_2["trk_contained"] ]
cc = var_Prange[is_pi_pm & is_numu_cc & df_2["trk_contained"] ] - var_mcs[is_pi_pm & is_numu_cc & df_2["trk_contained"] ]
dd = var_Prange[is_pi0 & is_numu_cc & df_2["trk_contained"] ] - var_mcs[is_pi0 & is_numu_cc & df_2["trk_contained"] ]
ee = var_Prange[is_elec_prot & is_numu_cc & df_2["trk_contained"] ] - var_mcs[is_elec_prot & is_numu_cc & df_2["trk_contained"] ]
ff = var_Prange[is_muon_antim & is_numu_cc & df_2["trk_contained"] ] - var_mcs[is_muon_antim & is_numu_cc & df_2["trk_contained"] ]
gg = var_Prange[is_gamma & is_numu_cc & df_2["trk_contained"] ] - var_mcs[is_gamma & is_numu_cc & df_2["trk_contained"] ]

colors = ['olivedrab', 'cyan', 'steelblue', 'magenta', 'yellow', 'tomato' ,'green']
names = ['proton', 'neutron', '$\pi+-$', '$\pi^{0}$', 'e+-', '$\mu+-$', '$\gamma$']
plt.hist([aa, bb, cc, dd, ee, ff, gg], color=colors, bins = bins, label=names, density=False, stacked = True)


#_ = plt.hist(var[is_cosmic & df["trk_contained"]], bins=bins, histtype="step", label="Cosmic")
plt.legend()
plt.xlabel("Reco Muon Momentum (Contained) with no cut on L - per particle type [GeV/c]")
plt.ylabel("# events")
plt.title("Momentum by range and MCS difference events (no cut on L)")
plt.show()




In [ ]:
#############  primary events from primary vertex  ########################

In [ ]:
####### Plotting #events with momentum (Contained) : with no cuts on L ########


# Now we can compute more stuff! Like the primary track momentum
###### calculating muon momentum (contained) :: Reco ############
is_numu_cc = (np.abs(df_2["rec.mc.nu.pdg"]) == 14) & (df_2["rec.mc.nu.iscc"])
is_primary_vertex = df_2["rec.slc.reco.trk.pfp.parent_is_primary"]      #### the primary event coming from primary interaction vertex
is_proton = df_2["rec.slc.reco.trk.truth.p.pdg"] == 2212
is_neutron = df_2["rec.slc.reco.trk.truth.p.pdg"] == 2112
is_pi_pm = np.abs(df_2["rec.slc.reco.trk.truth.p.pdg"]) == 211
is_pi0 = df_2["rec.slc.reco.trk.truth.p.pdg"] == 111
is_elec_prot = df_2["rec.slc.reco.trk.truth.p.pdg"] == 11      #### electron positron
is_muon_antim = df_2["rec.slc.reco.trk.truth.p.pdg"] == 13
is_gamma = df_2["rec.slc.reco.trk.truth.p.pdg"] == 22
is_cosmic = (df_2["rec.slc.tmatch.index"] < 0)

#px = df["rec.slc.truth.momentum.x"]
#py = df["rec.slc.truth.momentum.y"]
#pz = df["rec.slc.truth.momentum.z"]

#momentum_square= px**2 + py**2 + pz**2
#total_momentum = np.sqrt(momentum_square)


var = df_2["rec.slc.reco.trk.rangeP.p_muon"]
#var = df["rec.slc.truth.momentum.x"] + df["rec.slc.truth.momentum.y"] + df["rec.slc.truth.momentum.z"]

plt.figure()
bins = np.linspace(0, 2, 25)
#_ = plt.hist(var[is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\nu_\mu$ CC")
#_ = plt.hist(var[is_proton & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"proton", stacked=True)
#_ = plt.hist(var[is_neutron & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"neutron", stacked=True)
#_ = plt.hist(var[is_pi_pm & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\pi+-$", stacked=True)
#_ = plt.hist(var[is_pi0 & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\pi^{0}$", stacked=True)
#_ = plt.hist(var[is_elec_prot & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"e+-", stacked=True)
#_ = plt.hist(var[is_muon_antim & is_numu_cc & is_primary_vertex & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\mu+-$", stacked=True)
#_ = plt.hist(var[is_gamma & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\gamma$", stacked=True)
#_ = plt.hist(var[is_cosmic & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\cosmic$", stacked=True)

aa = var[is_proton & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
bb = var[is_neutron & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
cc = var[is_pi_pm & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
dd = var[is_pi0 & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
ee = var[is_elec_prot & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
ff = var[is_muon_antim & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
gg = var[is_gamma & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]

colors = ['olivedrab', 'cyan', 'steelblue', 'magenta', 'yellow', 'tomato' ,'green']
names = ['proton', 'neutron', '$\pi+-$', '$\pi^{0}$', 'e+-', '$\mu+-$', '$\gamma$']
plt.hist([aa, bb, cc, dd, ee, ff, gg], color=colors, bins = bins, label=names, density=False, stacked = True)



#_ = plt.hist(var[is_cosmic & df["trk_contained"]], bins=bins, histtype="step", label="Cosmic")
plt.legend()
plt.xlabel("Reco Muon Momentum (Contained : from primary vertex) - with no cut on L \n - per particle type [GeV/c]")
plt.ylabel("# events")
plt.title("From primary interaction vertex")
plt.show()

In [ ]:
####### Plotting #events with momentum (Contained) : with no cuts on L ########  INCLUDING COSMICS and primary vertex


# Now we can compute more stuff! Like the primary track momentum
###### calculating muon momentum (contained) :: Reco ############
is_numu_cc = (np.abs(df_2["rec.mc.nu.pdg"]) == 14) & (df_2["rec.mc.nu.iscc"])
is_primary_vertex = df_2["rec.slc.reco.trk.pfp.parent_is_primary"]      #### the primary event coming from primary interaction vertex
is_proton = df_2["rec.slc.reco.trk.truth.p.pdg"] == 2212
is_neutron = df_2["rec.slc.reco.trk.truth.p.pdg"] == 2112
is_pi_pm = np.abs(df_2["rec.slc.reco.trk.truth.p.pdg"]) == 211
is_pi0 = df_2["rec.slc.reco.trk.truth.p.pdg"] == 111
is_elec_prot = df_2["rec.slc.reco.trk.truth.p.pdg"] == 11      #### electron positron
is_muon_antim = df_2["rec.slc.reco.trk.truth.p.pdg"] == 13
is_gamma = df_2["rec.slc.reco.trk.truth.p.pdg"] == 22
is_cosmic = (df_2["rec.slc.tmatch.index"] < 0)

#px = df["rec.slc.truth.momentum.x"]
#py = df["rec.slc.truth.momentum.y"]
#pz = df["rec.slc.truth.momentum.z"]

#momentum_square= px**2 + py**2 + pz**2
#total_momentum = np.sqrt(momentum_square)


var = df_2["rec.slc.reco.trk.rangeP.p_muon"]
#var = df["rec.slc.truth.momentum.x"] + df["rec.slc.truth.momentum.y"] + df["rec.slc.truth.momentum.z"]

plt.figure()
bins = np.linspace(0, 2, 25)
#_ = plt.hist(var[is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\nu_\mu$ CC")
#_ = plt.hist(var[is_proton & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"proton", stacked=True)
#_ = plt.hist(var[is_neutron & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"neutron", stacked=True)
#_ = plt.hist(var[is_pi_pm & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\pi+-$", stacked=True)
#_ = plt.hist(var[is_pi0 & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\pi^{0}$", stacked=True)
#_ = plt.hist(var[is_elec_prot & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"e+-", stacked=True)
#_ = plt.hist(var[is_muon_antim & is_numu_cc & is_primary_vertex & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\mu+-$", stacked=True)
#_ = plt.hist(var[is_gamma & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\gamma$", stacked=True)
#_ = plt.hist(var[is_cosmic & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\cosmic$", stacked=True)

#_ = plt.hist(var[is_cosmic & df["trk_contained"]], bins=bins, histtype="step", label="Cosmic")

aa = var[is_proton & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
bb = var[is_neutron & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
cc = var[is_pi_pm & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
dd = var[is_pi0 & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
ee = var[is_elec_prot & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
ff = var[is_muon_antim & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
gg = var[is_gamma & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
hh = var[is_cosmic & is_primary_vertex & df_2["trk_contained"] ]

colors = ['olivedrab', 'cyan', 'steelblue', 'magenta', 'yellow', 'tomato' ,'green', 'lightsteelblue']
names = ['proton', 'neutron', '$\pi+-$', '$\pi^{0}$', 'e+-', '$\mu+-$', '$\gamma$', 'cosmics']
plt.hist([aa, bb, cc, dd, ee, ff, gg, hh], color=colors, bins = bins, label=names, density=False, stacked = True)

plt.legend()
plt.xlabel("Reco Muon Momentum (Contained : from primary vertex) - with no cut on L \n - per particle type [GeV/c]")
plt.ylabel("# events")
plt.title("From primary interaction vertex")
plt.show()

In [ ]:
############## cos theta for no cut on L ##########

In [ ]:
is_numu_cc = (np.abs(df_2["rec.mc.nu.pdg"]) == 14) & (df_2["rec.mc.nu.iscc"])
is_primary_vertex = df_2["rec.slc.reco.trk.pfp.parent_is_primary"]      #### the primary event coming from primary interaction vertex
is_proton = df_2["rec.slc.reco.trk.truth.p.pdg"] == 2212
is_neutron = df_2["rec.slc.reco.trk.truth.p.pdg"] == 2112
is_pi_pm = np.abs(df_2["rec.slc.reco.trk.truth.p.pdg"]) == 211
is_pi0 = df_2["rec.slc.reco.trk.truth.p.pdg"] == 111
is_elec_prot = df_2["rec.slc.reco.trk.truth.p.pdg"] == 11      #### electron positron
is_muon_antim = df_2["rec.slc.reco.trk.truth.p.pdg"] == 13
is_gamma = df_2["rec.slc.reco.trk.truth.p.pdg"] == 22
is_cosmic = (df_2["rec.slc.tmatch.index"] < 0)

var = df_2["rec.slc.reco.trk.costh"]

bins = np.linspace(-1, 1, 50)
_ = plt.hist(var[is_proton & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"proton", stacked=True)
_ = plt.hist(var[is_neutron & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"neutron", stacked=True)
_ = plt.hist(var[is_pi_pm & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\pi+-$", stacked=True)
_ = plt.hist(var[is_pi0 & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\pi^{0}$", stacked=True)
_ = plt.hist(var[is_elec_prot & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"e+-", stacked=True)
_ = plt.hist(var[is_muon_antim & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\mu+-$", stacked=True)
_ = plt.hist(var[is_gamma & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\gamma$", stacked=True)
_ = plt.hist(var[is_cosmic & is_primary_vertex &  df_2["trk_contained"] ], bins=bins, histtype="step", label=r"cosmics", stacked=True)

#plt.hist(df["rec.slc.reco.trk.costh"], bins=bins1, color = "m")
plt.xlabel("cos $\\theta$ (contained)")
plt.ylabel("# events")
plt.title("Plot of cos $\\theta$ per particle type including cosmics \n (contained) : with primary vertex - for no cut on L")
plt.legend()
plt.show()


In [ ]:
is_numu_cc = (np.abs(df_2["rec.mc.nu.pdg"]) == 14) & (df_2["rec.mc.nu.iscc"])
is_primary_vertex = df_2["rec.slc.reco.trk.pfp.parent_is_primary"]      #### the primary event coming from primary interaction vertex
is_proton = df_2["rec.slc.reco.trk.truth.p.pdg"] == 2212
is_neutron = df_2["rec.slc.reco.trk.truth.p.pdg"] == 2112
is_pi_pm = np.abs(df_2["rec.slc.reco.trk.truth.p.pdg"]) == 211
is_pi0 = df_2["rec.slc.reco.trk.truth.p.pdg"] == 111
is_elec_prot = df_2["rec.slc.reco.trk.truth.p.pdg"] == 11      #### electron positron
is_muon_antim = df_2["rec.slc.reco.trk.truth.p.pdg"] == 13
is_gamma = df_2["rec.slc.reco.trk.truth.p.pdg"] == 22
is_cosmic = (df_2["rec.slc.tmatch.index"] < 0)

var = df_2["rec.slc.reco.trk.costh"]

bins = np.linspace(-1, 1, 50)
#_ = plt.hist(var[is_numu_cc], bins=bins, histtype="step", label=r"$\nu_\mu$ CC")
#_ = plt.hist(var[is_cosmic], bins=bins, histtype="step", label=r"$\nu_\mu$ CC")

aa = var[is_proton & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
bb = var[is_neutron & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
cc = var[is_pi_pm & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
dd = var[is_pi0 & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
ee = var[is_elec_prot & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
ff = var[is_muon_antim & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
gg = var[is_gamma & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
hh = var[is_cosmic & is_primary_vertex & df_2["trk_contained"] ]

colors = ['olivedrab', 'cyan', 'steelblue', 'magenta', 'yellow', 'tomato' ,'green', 'lightsteelblue']
names = ['proton', 'neutron', '$\pi+-$', '$\pi^{0}$', 'e+-', '$\mu+-$', '$\gamma$', 'cosmics']
plt.hist([aa, bb, cc, dd, ee, ff, gg, hh], color=colors, bins = bins, label=names, density=False, stacked = True)



#plt.hist(df["rec.slc.reco.trk.costh"], bins=bins1, color = "m")
plt.xlabel("cos $\\theta$ (contained)")
plt.ylabel("# events")
plt.legend()
plt.title("Plot of cos $\\theta$ per particle type including cosmics \n (contained) - for no cut on L")
plt.show()

In [ ]:
############### angle phi plot for no cut on L ####################################

In [ ]:
is_numu_cc = (np.abs(df_2["rec.mc.nu.pdg"]) == 14) & (df_2["rec.mc.nu.iscc"])
is_primary_vertex = df_2["rec.slc.reco.trk.pfp.parent_is_primary"]      #### the primary event coming from primary interaction vertex
is_proton = df_2["rec.slc.reco.trk.truth.p.pdg"] == 2212
is_neutron = df_2["rec.slc.reco.trk.truth.p.pdg"] == 2112
is_pi_pm = np.abs(df_2["rec.slc.reco.trk.truth.p.pdg"]) == 211
is_pi0 = df_2["rec.slc.reco.trk.truth.p.pdg"] == 111
is_elec_prot = df_2["rec.slc.reco.trk.truth.p.pdg"] == 11      #### electron positron
is_muon_antim = df_2["rec.slc.reco.trk.truth.p.pdg"] == 13
is_gamma = df_2["rec.slc.reco.trk.truth.p.pdg"] == 22
is_cosmic = (df_2["rec.slc.tmatch.index"] < 0)

var = df_2["rec.slc.reco.trk.phi"]

bins = np.linspace(0, 2*math.pi, 50)
_ = plt.hist(var[is_proton & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"proton", stacked=True)
_ = plt.hist(var[is_neutron & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"neutron", stacked=True)
_ = plt.hist(var[is_pi_pm & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\pi+-$", stacked=True)
_ = plt.hist(var[is_pi0 & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\pi^{0}$", stacked=True)
_ = plt.hist(var[is_elec_prot & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"e+-", stacked=True)
_ = plt.hist(var[is_muon_antim & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\mu+-$", stacked=True)
_ = plt.hist(var[is_gamma & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ], bins=bins, histtype="step", label=r"$\gamma$", stacked=True)
_ = plt.hist(var[is_cosmic & is_primary_vertex &  df_2["trk_contained"] ], bins=bins, histtype="step", label=r"cosmics", stacked=True)

#plt.hist(df["rec.slc.reco.trk.costh"], bins=bins1, color = "m")
plt.xlabel("$\phi$ (contained)")
plt.ylabel("# events")
plt.title("Plot of angle $\phi$ per particle type including cosmics \n (contained): with primary vertex - for no cut on L")
plt.legend()
plt.show()



In [ ]:
is_numu_cc = (np.abs(df_2["rec.mc.nu.pdg"]) == 14) & (df_2["rec.mc.nu.iscc"])
is_primary_vertex = df_2["rec.slc.reco.trk.pfp.parent_is_primary"]      #### the primary event coming from primary interaction vertex
is_proton = df_2["rec.slc.reco.trk.truth.p.pdg"] == 2212
is_neutron = df_2["rec.slc.reco.trk.truth.p.pdg"] == 2112
is_pi_pm = np.abs(df_2["rec.slc.reco.trk.truth.p.pdg"]) == 211
is_pi0 = df_2["rec.slc.reco.trk.truth.p.pdg"] == 111
is_elec_prot = df_2["rec.slc.reco.trk.truth.p.pdg"] == 11      #### electron positron
is_muon_antim = df_2["rec.slc.reco.trk.truth.p.pdg"] == 13
is_gamma = df_2["rec.slc.reco.trk.truth.p.pdg"] == 22
is_cosmic = (df_2["rec.slc.tmatch.index"] < 0)

var = df_2["rec.slc.reco.trk.phi"]

bins = np.linspace(0, 2*math.pi, 50)
#_ = plt.hist(var[is_numu_cc], bins=bins, histtype="step", label=r"$\nu_\mu$ CC")
#_ = plt.hist(var[is_cosmic], bins=bins, histtype="step", label=r"$\nu_\mu$ CC")

aa = var[is_proton & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
bb = var[is_neutron & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
cc = var[is_pi_pm & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
dd = var[is_pi0 & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
ee = var[is_elec_prot & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
ff = var[is_muon_antim & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
gg = var[is_gamma & is_numu_cc & is_primary_vertex & df_2["trk_contained"] ]
hh = var[is_cosmic & is_primary_vertex & df_2["trk_contained"] ]

colors = ['olivedrab', 'cyan', 'steelblue', 'magenta', 'yellow', 'tomato' ,'green', 'lightsteelblue']
names = ['proton', 'neutron', '$\pi+-$', '$\pi^{0}$', 'e+-', '$\mu+-$', '$\gamma$', 'cosmics']
plt.hist([aa, bb, cc, dd, ee, ff, gg, hh], color=colors, bins = bins, label=names, density=False, stacked = True)



#plt.hist(df["rec.slc.reco.trk.costh"], bins=bins1, color = "m")
plt.xlabel("$\phi$ (contained)")
plt.ylabel("# events")
plt.title("Plot of angle $\phi$ per particle type including cosmics \n (contained) - for no cut on L")
plt.legend()
plt.show()


